In [ ]:
import pandas as pd
import os, zipfile
from sklearn.model_selection import train_test_split

def prepareData():
    """ 
    讀入資料，並區分訓練資料與驗證資料
    Returns:
        train_df(DataFrame)   :從train取出用於訓練的資料 (90%)
        validate_df(DataFrame):從train取出用於驗證的資料 (10%)
    """
    # 將訓練資料跟測試資料解壓縮
    # 解壓縮的zip檔名
    data = ['train', 'test']

    # 在當前的目錄解壓縮train.zip、test.zip
    path = '../input/dogs-vs-cats-redux-kernels-edition/'
    for el in data:
        with zipfile.ZipFile(path + el + ".zip", "r") as z:
            z.extractall(".")

    # 使用檔名dog.x.jpg、cat.x.jpg，建立標籤1與0 
    # 取得train資料夾內的檔名，放入filenames
    filenames = os.listdir("./train")
    # 放置標籤的清單
    categories = [] 
    for filename in filenames: 
        # 分割檔名、取出最前頭的元素(dog/cat)
        # 將dog為1、cat為0設為標籤，放入category  
        category = filename.split('.')[0] 
        if category == 'dog': # 若為 dog，則加上標籤1  
            categories.append(1) 
        else: # 若為cat，則加上標籤0
            categories.append(0)

    # 對df的列filename放入檔名filename
    # 對列category放入標籤數值categories
    df = pd.DataFrame({'filename': filenames,
                       'category': categories})

    # 將訓練資料總數25000已隨機方式分割為90%跟10%、
    # 90%為用於訓練的資料、10%為用於驗證的資料
    train_df, validate_df = train_test_split(df, test_size=0.1)
    # 重新配置列的索引
    train_df = train_df.reset_index()
    validate_df = validate_df.reset_index()

    return train_df, validate_df

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def ImageDataGenerate(train_df, validate_df):
    """
    對圖像進行加工
    parameters:
        train_df(DataFrame)   : 從train取出用於訓練的資料(90%) 
        validate_df(DataFrame): 從train取出用於驗證的資料(10%) 
    Returns: 
        train(DirectoryIterator)     : 加工過後的訓練資料 
        valid(DirectoryIterator): 加工過後的驗證資料 
    """ 
    # 重新調整圖像尺寸
    img_width, img_height = 224, 224
    target_size = (img_width, img_height)
    # 批次大小
    batch_size = 16

    # 檔名的欄位名稱，標籤的欄位名稱
    x_col, y_col = 'filename', 'category'
    # 設定flow_from_dataframe()的class_mode數值 
    # 此範例為二元分類，設定值為'binary'
    class_mode = 'binary'

    # 建立Generator來加工圖像
    train_datagen = ImageDataGenerator(rotation_range=15,
                                       rescale=1./255,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest',
                                       width_shift_range=0.1,
                                       height_shift_range=0.1)

    # 當flow_from_dataframe()引數為class_mode = "binary"時，
    # 標籤(train_df中的y_col = 'category')該行須為文字格式
    # 因此需要將將1與0的數字轉換為文字
    train_df['category'] = train_df['category'].astype(str)

    # 使用Generator產生加工完的訓練資料
    train = train_datagen.flow_from_dataframe(train_df,
                                              "./train/",
                                              x_col=x_col,
                                              y_col=y_col,
                                              class_mode=class_mode, 
                                              target_size=target_size, 
                                              batch_size=batch_size,
                                              shuffle=False)
    
     # 使用Generator產生加工完的驗證資料
    valid_datagen = ImageDataGenerator(rescale=1./255)

    # 當flow_from_dataframe()引數為class_mode = "binary"時，
    # 標籤(train_df中的y_col = 'category')該行須為文字格式
    # 因此需要將將1與0的數字轉換為文字
    validate_df['category'] = validate_df['category'].astype(str)

    # 使用Generator產生加工完的驗證資料
    valid = valid_datagen.flow_from_dataframe(validate_df,
                                              "./train/",
                                              x_col=x_col,
                                              y_col=y_col,
                                              class_mode=class_mode, 
                                              target_size=target_size, 
                                              batch_size=batch_size,
                                              shuffle=False)

    # 傳回訓練資料與驗證資料
    return train, valid 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalMaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import LearningRateScheduler
import math

def train_FClayer(train_generator, validation_generator):
    """
    使用完成微調的VGG16進行訓練
    Returns: history(History物件)
    """
    # 取得圖像尺寸
    image_size = len(train_generator[0][0][0])
    # 將輸入資料的形狀改為Tuple
    input_shape = (image_size, image_size, 3)
    # 批次大小
    batch_size = len(train_generator[0][0])
    # 取得訓練資料的數量
    total_train = len(train_generator) * batch_size
    # 取得驗證資料的數量
    total_validate = len(validation_generator) * batch_size

    # 一併讀入VGG16模型跟預學習之參數
    pre_trained_model = VGG16(include_top=False,  # 不用VGG16的全連接層
                              weights='imagenet', # 運用以ImageNet訓練好的參數
                              input_shape=input_shape) # 輸入資料之形狀

    for layer in pre_trained_model.layers[:15]: # 凍結第1 ~ 第15層的參數
        layer.trainable = False
    for layer in pre_trained_model.layers[15:]: # 使第16層之後的參數可以更新
        layer.trainable = True

    # 建立Sequentual物件
    model = Sequential()

    # 加入VGG16模型
    model.add(pre_trained_model)

    # 對四維張量(batch_size, rows, cols, channels)套用池化演算法後
    # 拉平為二維張量(batch_size, channels) 
    model.add(GlobalMaxPooling2D())

    # 全連接層
    model.add(Dense(512,                # 神經元數量為512
                    activation='relu')) # 激活函數為 ReLU
    # 丟棄率50%
    model.add(Dropout(0.5))

    # 輸出層
    model.add(Dense(1,                     # 神經元數量為 1
                    activation='sigmoid')) # 激活函數為Sigmoid

    # 模型編譯 
    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizers.RMSprop(lr=1e-5), 
                  metrics=['accuracy']) 

    # 編譯後顯示概要 
    model.summary() 

    # 對學習率進行排程 
    def step_decay(epoch): 
        initial_lrate = 0.00001 # 學習率初始值 
        drop = 0.5              # 衰減率為 50%
        epochs_drop = 10.0      # 每10次訓練週期進行衰減
        lrate = initial_lrate * math.pow(drop,
                                         math.floor((epoch)/epochs_drop))
        return lrate

    # 回呼學習率
    lrate = LearningRateScheduler(step_decay)

    # 使用完成微調的模型進行訓練
    epochs = 40 # 訓練週期
    
    history = model.fit(train_generator,
                        epochs=epochs,
                        validation_data=validation_generator,
                        validation_steps=total_validate//batch_size,
                        steps_per_epoch=total_train//batch_size,
                        verbose=1,
                        callbacks=[lrate])

    # 傳回history
    return history


In [ ]:
# 取得處理的資料
train_df, validate_df = prepareData()
# 資料預處理
train, valid = ImageDataGenerate(train_df, validate_df)
# 使用完成微調的VGG16模型進行訓練
history = train_FClayer(train, valid)